# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [24]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *

In [25]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [26]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [27]:
get_files('./data/song_data/')[0]

'/home/whoami/Notebooks/Data Engineer NanoDegree/sparkify-rdbms/data/song_data/A/A/A/TRAAAPK128E0786D96.json'

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [28]:
song_files = get_files('./data/song_data/')

In [29]:
filepath = song_files[0]
filepath

'/home/whoami/Notebooks/Data Engineer NanoDegree/sparkify-rdbms/data/song_data/A/A/A/TRAAAPK128E0786D96.json'

In [30]:
df = pd.read_json(filepath, orient='index', typ='series')


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [31]:
selected_columns = ['song_id', 'title', 'artist_id', 'year', 'duration']
song_data = df[selected_columns]
song_data.values


array(['SOHKNRJ12A6701D1F8', 'Drop of Rain', 'AR10USD1187B99F3F1', 0,
       189.57016], dtype=object)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [32]:
cur.execute(song_table_insert, song_data.values)
conn.commit()

UniqueViolation: duplicate key value violates unique constraint "song_pkey"
DETAIL:  Key (song_id)=(SOHKNRJ12A6701D1F8) already exists.


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [33]:
selected_columns = ['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']
ideal_indexes = ['artist_id', 'name', 'location', 'latitude', 'longitude']
artist_data = df[selected_columns]


artist_data = pd.Series(data=artist_data.values, index=ideal_indexes)
artist_data.values


array(['AR10USD1187B99F3F1', 'Tweeterfriendly Music',
       'Burlington, Ontario, Canada', None, None], dtype=object)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [34]:
cur.execute(artist_table_insert, artist_data.values)
conn.commit()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [35]:
log_files = get_files('./data/log_data/')

In [36]:
filepath = log_files[10]

In [37]:
df = pd.read_json(filepath, orient='records', lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Kevin,M,0,Arellano,NaN,free,"Harrisburg-Carlisle, PA",GET,Home,1.540007e+12,514,None,200,1542069417796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
1,Fu,Logged In,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,514,Ja I Ty,200,1542069637796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
2,None,Logged In,Maia,F,0,Burke,NaN,free,"Houston-The Woodlands-Sugar Land, TX",GET,Home,1.540677e+12,510,None,200,1542071524796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",51
3,All Time Low,Logged In,Maia,F,1,Burke,177.84118,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540677e+12,510,A Party Song (The Walk of Shame),200,1542071549796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",51
4,Nik & Jay,Logged In,Wyatt,M,0,Scott,196.51873,free,"Eureka-Arcata-Fortuna, CA",PUT,NextSong,1.540872e+12,379,Pop-Pop!,200,1542079142796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [38]:
df = df[df.page == 'NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Fu,Logged In,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,514,Ja I Ty,200,1542069637796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
3,All Time Low,Logged In,Maia,F,1,Burke,177.84118,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540677e+12,510,A Party Song (The Walk of Shame),200,1542071549796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",51
4,Nik & Jay,Logged In,Wyatt,M,0,Scott,196.51873,free,"Eureka-Arcata-Fortuna, CA",PUT,NextSong,1.540872e+12,379,Pop-Pop!,200,1542079142796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9
5,Quad City DJ's,Logged In,Chloe,F,0,Cuevas,451.44771,free,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,506,C'mon N' Ride It (The Train) (LP Version),200,1542081112796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
8,Hoobastank,Logged In,Noah,M,1,Chavez,232.17587,free,"Ogden-Clearfield, UT",PUT,NextSong,1.540971e+12,492,Born To Lead,200,1542085206796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) G...,94


In [39]:
converted_timestamps = pd.to_datetime(df['ts'].values, unit='ms')


In [40]:

series = pd.Series(converted_timestamps)
time_data = {
    'start_time': df['ts'].values,
    'hour': list(series.dt.hour.values),
    'day': list(series.dt.day.values),
    'week': list(series.dt.week), 
    'month': list(series.dt.month), 
    'year': list(series.dt.year), 
    'weekday': list(series.dt.weekday),   
}
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']

In [41]:
time_df = pd.DataFrame(data=time_data, columns=column_labels)
time_df

,start_time,hour,day,week,month,year,weekday
0,1542069637796,0,13,46,11,2018,1
1,1542071549796,1,13,46,11,2018,1
2,1542079142796,3,13,46,11,2018,1
3,1542081112796,3,13,46,11,2018,1
4,1542085206796,5,13,46,11,2018,1
...,...,...,...,...,...,...,...
334,1542152631796,23,13,46,11,2018,1
335,1542152868796,23,13,46,11,2018,1
336,1542153200796,23,13,46,11,2018,1
337,1542153362796,23,13,46,11,2018,1


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [42]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [43]:
df.head()
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']].drop_duplicates(subset=['userId'])
user_df

,userId,firstName,lastName,gender,level
1,66,Kevin,Arellano,M,free
3,51,Maia,Burke,F,free
4,9,Wyatt,Scott,M,free
5,49,Chloe,Cuevas,F,free
8,94,Noah,Chavez,M,free
9,60,Devin,Larson,M,free
13,25,Jayden,Graves,M,paid
36,83,Stefany,White,F,free
44,44,Aleena,Kirby,F,paid
49,97,Kate,Harrell,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [44]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [48]:
for index, row in df.iterrows():
    # get songid and artistid from song and artist tables
    print(row)
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
     
    if results:
        songid, artistid = results[0], results[2]
    else:
        songid, artistid = None, None
    
    #print(songid, artistid)
    print(row, row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location)
    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

artist                                                          Fu
auth                                                     Logged In
firstName                                                    Kevin
gender                                                           M
itemInSession                                                    1
lastName                                                  Arellano
length                                                     280.058
level                                                         free
location                                   Harrisburg-Carlisle, PA
method                                                         PUT
page                                                      NextSong
registration                                           1.54001e+12
sessionId                                                      514
song                                                       Ja I Ty
status                                                        

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [49]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.